# Capstone Project - The Battle of the Neighborhoods

In [2]:
import pandas as pd 
import numpy as np 
import random 
from bs4 import BeautifulSoup   
import geocoder
from geopy.geocoders import Nominatim 
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize
import folium 

print('Libraries imported.')

Libraries imported.


In [3]:
CLIENT_ID = 'AT0DE2SZOFMWQGY24TV4ENQCKSRG4Y4LZVPBTJWUGUP03SNB' # your Foursquare ID
CLIENT_SECRET = 'DK55FF5FF1F1P5ZBGKP5TRBBW53NGOPOSFMJX2URJKOKD5XR' # your Foursquare Secret

VERSION = '20200601'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AT0DE2SZOFMWQGY24TV4ENQCKSRG4Y4LZVPBTJWUGUP03SNB
CLIENT_SECRET:DK55FF5FF1F1P5ZBGKP5TRBBW53NGOPOSFMJX2URJKOKD5XR


In [4]:
df=pd.read_csv('london_crime_by_lsoa.csv')
df.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01001116,Croydon,Burglary,Burglary in Other Buildings,0,2016,11
1,E01001646,Greenwich,Violence Against the Person,Other violence,0,2016,11
2,E01000677,Bromley,Violence Against the Person,Other violence,0,2015,5
3,E01003774,Redbridge,Burglary,Burglary in Other Buildings,0,2016,3
4,E01004563,Wandsworth,Robbery,Personal Property,0,2008,6


In [5]:
df.drop(df.index[df['year'] != 2016], inplace = True)

# Removing all the entires where crime values are null  
df = df[df.value != 0]

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

In [6]:
df.shape

(392042, 7)

In [7]:
df.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01004177,Sutton,Theft and Handling,Theft/Taking of Pedal Cycle,1,2016,8
1,E01000733,Bromley,Criminal Damage,Criminal Damage To Motor Vehicle,1,2016,4
2,E01003989,Southwark,Theft and Handling,Theft From Shops,4,2016,8
3,E01002276,Havering,Burglary,Burglary in a Dwelling,1,2016,8
4,E01003674,Redbridge,Drugs,Possession Of Drugs,2,2016,11


In [8]:
#Changing the names of columns
df.columns = ['LSOA_Code', 'Borough','Major_Category','Minor_Category','No_of_Crimes','Year','Month']
df.head()

,LSOA_Code,Borough,Major_Category,Minor_Category,No_of_Crimes,Year,Month
0,E01004177,Sutton,Theft and Handling,Theft/Taking of Pedal Cycle,1,2016,8
1,E01000733,Bromley,Criminal Damage,Criminal Damage To Motor Vehicle,1,2016,4
2,E01003989,Southwark,Theft and Handling,Theft From Shops,4,2016,8
3,E01002276,Havering,Burglary,Burglary in a Dwelling,1,2016,8
4,E01003674,Redbridge,Drugs,Possession Of Drugs,2,2016,11


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392042 entries, 0 to 392041
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   LSOA_Code       392042 non-null  object
 1   Borough         392042 non-null  object
 2   Major_Category  392042 non-null  object
 3   Minor_Category  392042 non-null  object
 4   No_of_Crimes    392042 non-null  int64 
 5   Year            392042 non-null  int64 
 6   Month           392042 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 20.9+ MB


In [10]:
#Total number of crimes in borough
df['Borough'].value_counts()

Lambeth                   17605
Southwark                 16560
Croydon                   16254
Newham                    15622
Ealing                    15284
Tower Hamlets             15219
Brent                     14980
Barnet                    14668
Hackney                   14392
Lewisham                  14235
Haringey                  14202
Enfield                   13658
Wandsworth                13498
Westminster               13383
Islington                 13116
Greenwich                 12750
Camden                    12632
Hillingdon                12417
Hounslow                  12316
Waltham Forest            12121
Bromley                   11980
Redbridge                 11490
Hammersmith and Fulham    10281
Barking and Dagenham       9784
Havering                   9699
Kensington and Chelsea     9653
Harrow                     8257
Bexley                     8245
Merton                     8223
Richmond upon Thames       7199
Sutton                     6823
Kingston

In [11]:
#Total crimes in major city
df['Major_Category'].value_counts()

Theft and Handling             129159
Violence Against the Person    123050
Criminal Damage                 48584
Burglary                        43020
Drugs                           21782
Robbery                         14889
Other Notifiable Offences       11558
Name: Major_Category, dtype: int64

### Pivoting the table to view the no. of crimes for each major category in each Borough

In [13]:
London_crime = pd.pivot_table(df,values=['No_of_Crimes'], index=['Borough'],columns=['Major_Category'],aggfunc=np.sum,fill_value=0)
London_crime.head()

No_of_Crimes                        \
Major_Category           Burglary Criminal Damage Drugs   
Borough                                                   
Barking and Dagenham         1287            1949   919   
Barnet                       3402            2183   906   
Bexley                       1123            1673   646   
Brent                        2631            2280  2096   
Bromley                      2214            2202   728   

                                                                           \
Major_Category       Other Notifiable Offences Robbery Theft and Handling   
Borough                                                                     
Barking and Dagenham                       378     534               5607   
Barnet                                     499     464               9731   
Bexley                                     294     209               4392   
Brent                                      536     919               9026   
Bromley                                    417     369               7584   

                                                  
Major_Category       Violence Against the Person  
Borough                                           
Barking and Dagenham                        6067  
Barnet                                      7499  
Bexley                                      4503  
Brent                                       9205  
Bromley                                     6650

In [15]:
London_crime.reset_index(inplace = True)
London_crime['Total'] = London_crime.sum(axis=1)
London_crime.head(30)

index                 Borough No_of_Crimes                  \
Major_Category                                   Burglary Criminal Damage   
0                  0    Barking and Dagenham         1287            1949   
1                  1                  Barnet         3402            2183   
2                  2                  Bexley         1123            1673   
3                  3                   Brent         2631            2280   
4                  4                 Bromley         2214            2202   
5                  5                  Camden         2652            1935   
6                  6          City of London            2               2   
7                  7                 Croydon         2738            3219   
8                  8                  Ealing         2492            2562   
9                  9                 Enfield         2541            2136   
10                10               Greenwich         1780            2476   
11                11                 Hackney         2719            1981   
12                12  Hammersmith and Fulham         1531            1408   
13                13                Haringey         2474            2233   
14                14                  Harrow         1994            1212   
15                15                Havering         1826            1804   
16                16              Hillingdon         2229            2470   
17                17                Hounslow         1808            2213   
18                18               Islington         2051            1903   
19                19  Kensington and Chelsea         1449            1051   
20                20    Kingston upon Thames          879            1054   
21                21                 Lambeth         3087            2764   
22                22                Lewisham         2071            2351   
23                23                  Merton         1419            1418   
24                24                  Newham         2115            2496   
25                25               Redbridge         1997            1650   
26                26    Richmond upon Thames         1359            1148   
27                27               Southwark         2946            2621   
28                28                  Sutton         1233            1316   
29                29           Tower Hamlets         2794            2357   

                                                                           \
Major_Category Drugs Other Notifiable Offences Robbery Theft and Handling   
0                919                       378     534               5607   
1                906                       499     464               9731   
2                646                       294     209               4392   
3               2096                       536     919               9026   
4                728                       417     369               7584   
5               1493                       490     899              14088   
6                 10                         6       4                129   
7               1367                       718    1139               9229   
8               1355                       613     669              10040   
9               1063                       492     807               8037   
10               867                       521     486               8010   
11              1353                       499    1030              11851   
12              1321                       474     397               8925   
13              1540                       551    1248              10554   
14               473                       267     377               4537   
15               718                       389     311               5919   
16              1042                      1305     420               9045   
17              1086                       665     392               8682   
18        

In [16]:
London_crime.columns = London_crime.columns.map(''.join)
London_crime.head()

,index,Borough,No_of_CrimesBurglary,No_of_CrimesCriminal Damage,No_of_CrimesDrugs,No_of_CrimesOther Notifiable Offences,No_of_CrimesRobbery,No_of_CrimesTheft and Handling,No_of_CrimesViolence Against the Person,Total
0,0,Barking and Dagenham,1287,1949,919,378,534,5607,6067,33482
1,1,Barnet,3402,2183,906,499,464,9731,7499,49369
2,2,Bexley,1123,1673,646,294,209,4392,4503,25682
3,3,Brent,2631,2280,2096,536,919,9026,9205,53389
4,4,Bromley,2214,2202,728,417,369,7584,6650,40332


In [18]:
London_crime.columns = ['Index','Borough','Burglary', 'Criminal Damage','Drugs','Other Notifiable Offences',
                        'Robbery','Theft and Handling','Violence Against the Person','Total']
London_crime.head()

,Index,Borough,Burglary,Criminal Damage,Drugs,Other Notifiable Offences,Robbery,Theft and Handling,Violence Against the Person,Total
0,0,Barking and Dagenham,1287,1949,919,378,534,5607,6067,33482
1,1,Barnet,3402,2183,906,499,464,9731,7499,49369
2,2,Bexley,1123,1673,646,294,209,4392,4503,25682
3,3,Brent,2631,2280,2096,536,919,9026,9205,53389
4,4,Bromley,2214,2202,728,417,369,7584,6650,40332


In [34]:
London_crime=London_crime.drop(['Index'], axis=1)
#df.drop(['A'], axis = 1)
London_crime.head()

,Borough,Burglary,Criminal Damage,Drugs,Other Notifiable Offences,Robbery,Theft and Handling,Violence Against the Person,Total
0,Barking and Dagenham,1287,1949,919,378,534,5607,6067,33482
1,Barnet,3402,2183,906,499,464,9731,7499,49369
2,Bexley,1123,1673,646,294,209,4392,4503,25682
3,Brent,2631,2280,2096,536,919,9026,9205,53389
4,Bromley,2214,2202,728,417,369,7584,6650,40332


In [35]:
London_crime.columns.tolist()

['Borough',
 'Burglary',
 'Criminal Damage',
 'Drugs',
 'Other Notifiable Offences',
 'Robbery',
 'Theft and Handling',
 'Violence Against the Person',
 'Total']

In [36]:
London_crime.shape

(33, 9)

## Part 2: Scraping additional information of the different Boroughs in London